In [2]:
import torch
from torch.utils import data
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
plt.style.use('seaborn') 
matplotlib.rcParams['font.family'] = 'serif'  
os.environ['KMP_DUPLICATE_LIB_OK']='True' # Avoid iOS bug with pyplot. 


# Data loading
from utils.dlc_practical_prologue import generate_pair_sets



# Import models
from models.LeNet import lenet
from models.LinearReluNet import linear_relu_net
from models.SimpleConvNet import simple_conv_net
from models.ResNet import residual_net


# Import plot utils
from utils.plot_utils import *
from utils.ml_utils import *
from utils.data_utils import *
from utils.metrics import accuracy

In [3]:
N = 1000  # Number of pairs
NN_ARGS = {"input_size": 14 * 14, "num_classes": 10}
DROP = {"d1" : 0, "d2" : 0}
DROP_RES = {"d" : 0.15}
BATCH_SIZE = 10
NUM_EPOCHS = 25
NUM_ITER = 1
MODELS = {
    "SimpleConvNet" : {"args" : DROP , "f" :simple_conv_net, "flatten": False, "name":"Simple Conv" },
    "LinearRelu": {"args": NN_ARGS, "f": linear_relu_net, "flatten": True, "name":"Linear ReLu"},
    "LeNet": {"args": DROP, "f": lenet, "flatten": False, "name":"LeNet"},
    "ResNet": {"args": DROP_RES, "f": residual_net, "flatten": False, "name":"Residual Net"},
}

In [8]:
def full_train_test(model_info, num_iter, num_epochs, data_size, batch_size, verbose=False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Copy all model parameters to the GPU
    
    args = model_info.get('args')
    if args is None:
        model = model_info.get('f')()
    else:
        model = model_info.get('f')(**args)

    model = model.to(device)
    
    all_train_losses = [] 
    all_test_losses = []
    
    all_test_accuracies = []
    
    for it in range(1, num_iter + 1):
        print("Iteration %d" % it)
        
        # Generate a new dataset
        train_input, train_target, train_classes, test_input, test_target, test_classes = generate_pair_sets(data_size)
        
        # Build Keras loaders for train and test datasets batches.
        tr_loader, te_loader = build_train_loaders(train_input, train_classes, test_input, test_classes, batch_size)
        
        tr_losses, te_losses = train_model(model, device, num_epochs, tr_loader, te_loader, model_info.get('flatten'), verbose)
        
       # plot_train_test(tr_losses, te_losses, model_info.get('name'))
    
        if verbose:
            print("Final train loss: %.4f" % tr_losses[-1])
            print("Final test loss: %.4f" % te_losses[-1])
    
        all_train_losses.append(tr_losses)
        all_test_losses.append(te_losses)
        
        _,_,_, acc_input, acc_target, _ = generate_pair_sets(data_size)
        acc_loader = build_test_loader(acc_input, acc_target, batch_size)
        
        acc = accuracy(model, acc_loader, None, model_info.get('flatten'))
        if verbose:
            print("Test accuracy: %.4f" % acc)
        all_test_accuracies.append(acc)
    
    
    plot_losses(all_train_losses, all_test_losses, model_info.get('name'))
    
    plot_accuracy(all_test_accuracies, model_info.get('name'))
    
    
    return all_train_losses, all_test_losses, all_test_accuracies

In [7]:
all_tr_losses, all_te_losses, all_te_acc = full_train_test(MODELS.get("SimpleConvNet"), NUM_ITER, NUM_EPOCHS, N, BATCH_SIZE)

Iteration 1


NameError: name 'model1' is not defined